In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf

1. Importing FastAPI, File, UploadFile:
- *from fastapi import FastAPI, File, UploadFile*: This line imports the necessary components from the FastAPI framework. FastAPI is a modern web framework for building APIs with Python.

2. Importing CORSMiddleware:
- *from fastapi.middleware.cors import CORSMiddleware*: This line imports the CORS (Cross-Origin Resource Sharing) middleware from FastAPI. CORS is used to enable cross-origin requests, allowing your API to be accessed by web applications running on different domains.

3. Importing uvicorn:
- import uvicorn: Uvicorn is an ASGI (Asynchronous Server Gateway Interface) server, commonly used for running FastAPI applications. It's used later in the code to start the server.

4. Importing numpy and other libraries:
- import numpy as np: NumPy is a popular library for numerical operations in Python. It's often used in machine learning for data manipulation.
- from io import BytesIO: BytesIO is an in-memory binary stream used for handling binary data.
- from PIL import Image: PIL (Python Imaging Library) is used for image processing. It's commonly used for opening, manipulating, and saving image files.
- import tensorflow as tf: TensorFlow is a popular machine learning library. In this context, it is likely used for loading a pre-trained machine learning model for inference.

Now, let's briefly explain the purpose of the code:

* FastAPI is used to create a web application.
* CORSMiddleware is used to enable Cross-Origin Resource Sharing, allowing your API to be accessed from different domains.
* uvicorn is a server that can run the FastAPI application.
* numpy is often used for data manipulation, possibly for processing data in the context of the machine learning model.
* BytesIO is used for handling binary data, which may be relevant for processing image data.
* PIL is used for image manipulation, and Image is a class for working with images.
* tensorflow is used for machine learning tasks and may be used for loading and running a pre-trained model.
* The code you provided is just the import section of a FastAPI application. To understand the full functionality of the API and how these packages are used, you would need to see the rest of the code, which would include the actual API endpoints and model serving logic.

In [ ]:
app = FastAPI()

The line app = FastAPI() creates a FastAPI application instance named app. This instance will be used to define and configure your API's endpoints, middleware, request handlers, and other settings.

With this instance, you can start building your API by adding routes, request handlers, and other components. For example, you can define routes that handle incoming HTTP requests, specify data models for request and response objects, add middleware for authentication or CORS handling, and more.

In [ ]:
origins = [
    "http://localhost",
    "http://localhost:3000",
]

The origins list is a configuration setting used with the CORSMiddleware in a FastAPI application. This setting defines a list of allowed origins (i.e., domains or URLs) that are permitted to make cross-origin requests to your API. Cross-origin requests occur when a web application running on one domain tries to access resources on a different domain through AJAX or other HTTP methods.

In this specific case, the origins list allows requests from the following two origins:

"http://localhost": This allows requests from a web application running on the local machine at the default HTTP port (port 80). Web applications running on "http://localhost" are often used for development and testing purposes.

"http://localhost:3000": This allows requests from a web application running on the local machine at port 3000. This is another common setup for development and testing, especially if you are using tools like React or other front-end frameworks that use this port by default.

By specifying these origins in the origins list, you are configuring your FastAPI application to accept requests from these specific domains. Any requests coming from other origins will be denied by default, helping to prevent potential security vulnerabilities related to cross-origin requests.

You would typically use this configuration when setting up the CORSMiddleware in your FastAPI application to control which domains are allowed to access your API.

In [ ]:
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

The code you provided adds the CORSMiddleware to your FastAPI application (app) with specific configuration options. Let's break down what each of these options does:

allow_origins=origins: This option specifies the list of allowed origins (domains or URLs) that are permitted to make cross-origin requests to your API. In your case, it allows requests from the origins specified in the origins list, which are "http://localhost" and "http://localhost:3000".

allow_credentials=True: This option allows the inclusion of credentials (e.g., cookies, HTTP authentication) when making cross-origin requests. If set to True, it indicates that the client-side JavaScript code running in the browser can include credentials when making requests to your API.

allow_methods=["*"]: This option specifies which HTTP methods are allowed for cross-origin requests. The value ["*"] means that all HTTP methods are allowed, which includes common methods like GET, POST, PUT, DELETE, etc. You can restrict this to specific methods if needed.

allow_headers=["*"]: This option specifies which HTTP headers are allowed for cross-origin requests. Similar to allow_methods, ["*"] means that all headers are allowed. You can restrict this to specific headers if needed. Allowing all headers might be acceptable for development or internal use, but for production APIs, it's a good practice to be more specific about which headers are allowed.

By adding the CORSMiddleware with these options to your FastAPI application, you are configuring it to handle cross-origin requests from the specified origins, allow credentials to be included in requests, and permit all HTTP methods and headers. This configuration is commonly used when you want to make your API accessible to web applications running on different domains while ensuring security and control over the allowed origins and request handling.

In [ ]:
MODEL = tf.keras.models.load_model("../saved_models/1")

The line MODEL = tf.keras.models.load_model("../saved_models/1") is loading a pre-trained machine learning model using TensorFlow's Keras API.

In [ ]:
CLASS_NAMES = ["Early Blight", "Late Blight", "Healthy"]

In [ ]:
@app.get("/ping")
async def ping():
    return "Hello, I am alive"

In [ ]:
def read_file_as_image(data) -> np.ndarray:
    image = np.array(Image.open(BytesIO(data)))
    return image

In [ ]:
@app.post("/predict")
async def predict(
    file: UploadFile = File(...)
):
    image = read_file_as_image(await file.read())
    img_batch = np.expand_dims(image, 0)
    
    predictions = MODEL.predict(img_batch)

    predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
    confidence = np.max(predictions[0])
    return {
        'class': predicted_class,
        'confidence': float(confidence)
    }

In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host='localhost', port=8000)